In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Define the CNN model architecture
def create_model(learning_rate, batch_size, num_filters, num_layers):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(num_filters, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    for _ in range(num_layers - 1):
        model.add(tf.keras.layers.Conv2D(num_filters, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
# Define the objective function to evaluate the model
def objective_function(params):
    learning_rate, batch_size, num_filters, num_layers = params
    model = create_model(learning_rate, int(batch_size), int(num_filters), int(num_layers))
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=5, verbose=0)
    y_pred_prob = model.predict(X_val)
    y_pred = np.argmax(y_pred_prob, axis=1)
    accuracy = accuracy_score(y_val, y_pred)
    return -accuracy  # maximize accuracy


# PSO implementation
def pso_search(num_particles, num_iterations):
    particles = np.random.uniform(low=0, high=1, size=(num_particles, 4))
    velocities = np.zeros_like(particles)
    global_best = None
    global_best_fitness = np.inf

    for _ in range(num_iterations):
        for i in range(num_particles):
            # Ensure num_filters is at least 1
            num_filters = max(1, particles[i][2])
            particles[i][2] = num_filters

            fitness = objective_function(particles[i])
            if fitness < global_best_fitness:
                global_best = particles[i].copy()
                global_best_fitness = fitness
            best_particle = particles[i].copy()

            velocities[i] = 0.5 * velocities[i] + 2 * np.random.random() * (best_particle - particles[i]) \
                            + 2 * np.random.random() * (global_best - particles[i])
            particles[i] += velocities[i]

            particles[i] = np.clip(particles[i], 0, 1)

    return global_best

# Load and preprocess the dataset
(X_train_full, y_train_full), (_, _) = mnist.load_data()
X_train_full = X_train_full.reshape((-1, 28, 28, 1)) / 255.0
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Select a subset of the data for faster training
subset_size = 10000
subset_indices = np.random.choice(X_train.shape[0], size=subset_size, replace=False)
X_train_subset = X_train[subset_indices]
y_train_subset = y_train[subset_indices]






# Set the hyperparameters for PSO
num_particles = 20
num_iterations = 10

# Perform PSO search for hyperparameter optimization
best_hyperparameters = pso_search(num_particles, num_iterations)


# Train the CNN model with the best hyperparameters
learning_rate, batch_size, num_filters, num_layers = best_hyperparameters
best_model = create_model(learning_rate, int(batch_size), int(num_filters), int(num_layers))

num_epochs = 5
new_batch_size = 16  # Decrease the batch size

for epoch in range(num_epochs):
    best_model.fit(X_train, y_train, batch_size=new_batch_size, epochs=1, verbose=0)  # Train for one epoch
    loss, accuracy = best_model.evaluate(X_test, y_test)  # Evaluate on the test set
    print("Epoch:", epoch+1, "- Test Loss:", loss, "- Test Accuracy:", accuracy)


313/313 [==============================] - 3s 8ms/step - loss: 0.3844 - accuracy: 0.8832
Epoch: 1 - Test Loss: 0.38439127802848816 - Test Accuracy: 0.8831999897956848
313/313 [==============================] - 2s 5ms/step - loss: 0.2994 - accuracy: 0.9166
Epoch: 2 - Test Loss: 0.29936572909355164 - Test Accuracy: 0.9165999889373779
313/313 [==============================] - 2s 5ms/step - loss: 0.2936 - accuracy: 0.9118
Epoch: 3 - Test Loss: 0.29357093572616577 - Test Accuracy: 0.9118000268936157
313/313 [==============================] - 2s 7ms/step - loss: 0.3433 - accuracy: 0.9023
Epoch: 4 - Test Loss: 0.3432980179786682 - Test Accuracy: 0.9023000001907349
313/313 [==============================] - 3s 9ms/step - loss: 0.2953 - accuracy: 0.9154
Epoch: 5 - Test Loss: 0.29534053802490234 - Test Accuracy: 0.9154000282287598


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess the dataset
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()
X_train_full = X_train_full.reshape((-1, 28, 28, 1)) / 255.0
X_test = X_test.reshape((-1, 28, 28, 1)) / 255.0

# Select a subset of the data for faster training
subset_size = 10000
subset_indices = np.random.choice(X_train_full.shape[0], size=subset_size, replace=False)
X_train_subset = X_train_full[subset_indices]
y_train_subset = y_train_full[subset_indices]

# Split the training subset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=42)

# Define the CNN model architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
num_epochs = 5
batch_size = 16

for epoch in range(num_epochs):
    model.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=0)  # Train for one epoch
    _, accuracy = model.evaluate(X_val, y_val)  # Evaluate on the validation set
    print("Epoch:", epoch+1, "- Validation Accuracy:", accuracy)

# Evaluate the model on the test set
_, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)


63/63 [==============================] - 1s 16ms/step - loss: 0.1488 - accuracy: 0.9555
Epoch: 1 - Validation Accuracy: 0.9555000066757202
63/63 [==============================] - 1s 15ms/step - loss: 0.1174 - accuracy: 0.9660
Epoch: 2 - Validation Accuracy: 0.9660000205039978
63/63 [==============================] - 1s 9ms/step - loss: 0.1054 - accuracy: 0.9675
Epoch: 3 - Validation Accuracy: 0.9674999713897705
63/63 [==============================] - 1s 9ms/step - loss: 0.0947 - accuracy: 0.9735
Epoch: 4 - Validation Accuracy: 0.9735000133514404
63/63 [==============================] - 1s 9ms/step - loss: 0.0924 - accuracy: 0.9750
Epoch: 5 - Validation Accuracy: 0.9750000238418579
313/313 [==============================] - 3s 9ms/step - loss: 0.0779 - accuracy: 0.9777
Test Accuracy: 0.9776999950408936
